<h2><font color="#004D7F" size=5>Módulo 1: Modelos múltiples</font></h2>



<h1><font color="#004D7F" size=6> 1. Modelos One-vs-Rest y One-vs-One </font></h1>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Manuel Castillo-Cara</font><br>
<font color="#004D7F" size=3>Aprendizaje Automático II</font><br>
<font color="#004D7F" size=3>Universidad Nacional de Educación a Distancia</font>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>


* [1. Clasificadores binarios para clasificación multiclase](#section1)
* [2. Uno contra resto para clasificación multiclase](#section2)
    * [2.1. Dataset](#section21)
    * [2.2. OvR con LoR](#section22)
    * [2.3. OvR con Scikit-learn](#section23)
    * [2.4. OvR Predicciones](#section24)
* [3. Uno contra uno para clasificación multiclase](#section3)
    * [3.1. OvO en SVM](#section31)
    * [3.2. OvO con Scikit-learn](#section32)
    * [3.3. OvO Predicciones](#section33)
* [Ejercicios](#sectionEj)

---

<a id="section0"></a>
# <font color="#004D7F">0. Contexto</font>

No todos los modelos predictivos de clasificación admiten la clasificación multiclase. Un enfoque para utilizar algoritmos de clasificación binaria para problemas de clasificación de clases múltiples es usar las estrategias One-vs-Rest y One-vs-One. Después de completar este tutorial, sabrá: 

- Los modelos de clasificación binaria como la LoR y la SVM no admiten la clasificación multiclase de forma nativa y requieren metaestrategias.
- La estrategia Uno-vs-Rest divide una clasificación multiclase en un problema de clasificación binaria por clase.
- La estrategia One-vs-One divide una clasificación multiclase en un problema de clasificación binaria por cada par de clases.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section1"></a>
# <font color="#004D7F"> 1. Clasificadores binarios para clasificación multiclase</font>

Recordemos los dos tipos de un problema clasificación que se tiene:
- Clasificación Binaria: Tareas de clasificación con dos clases.
- Clasificación Multiclase: Tareas de clasificación con más de dos clases.

Algunos algoritmos están diseñados para problemas de clasificación binaria:
- Regresión logística
- Perceptrón
- Máquinas de vectores de soporte

Como tales, no se pueden utilizar para tareas de clasificación multiclase, al menos no directamente. En cambio,
se pueden utilizar métodos heurísticos para dividir un problema de clasificación multiclase en múltiples conjuntos de datos de clasificación binaria y entrenar un modelo de clasificación binaria para cada uno. Dos ejemplos de estos métodos heurísticos incluyen:
- Uno contra el resto (OvR)
- Uno contra uno (OvO)

<figure>
  <img src="datos/multiclass.png" width="350" height="350" alt="Gráfica">
  <figcaption><blockquote>Gráfico con los diferentes algoritmos multiclase y multioutput en Sklearn. Extraída de la documentación de Scikit-learn</blockquote></figcaption>
</figure>


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Más información__: Las diferentes clases para poder utilizar multiclase en scikit-learn se encuentran detalladas en la misma página de información de la librería. [Pulse aquí](https://qu4nt.github.io/sklearn-doc-es/modules/multiclass.html)
</div>

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section2"></a> 
# <font color="#004D7F"> 2. Uno contra resto para clasificación multiclase </font>

One-vs-Rest (OvR)es un método heurístico que
- Implica dividir el conjunto de datos multiclase en múltiples problemas de clasificación binaria.
- Luego se entrena un clasificador binario en cada problema de clasificación binaria y
- Se realizan predicciones utilizando el modelo que tenga más confianza. 

Por ejemplo, dado un problema de clasificación multiclase con ejemplos para cada clase rojo, azul y verde:
- Problema de clasificación binaria 1: rojo versus [azul, verde]
- Problema de clasificación binaria 2: azul versus [rojo, verde]
- Problema de clasificación binaria 3: verde versus [rojo, azul]

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Desventaja__: requiere la creación de un modelo para cada clase (tres clases requieren tres modelos). Esto podría ser un problema para conjuntos de datos grandes (por ejemplo, millones de filas), modelos lentos (por ejemplo, SVM) o números muy grandes de clases (por ejemplo, cientos de clases). 
</div>

<a id="section21"></a> 
## <font color="#004D7F"> 2.1. Dataset </font>

Como tal, la implementación de estos algoritmos en la biblioteca scikit-learn implementa la estrategia OvR de forma predeterminada cuando se utilizan estos algoritmos para la clasificación multiclase. 
1. Primero, definamos un conjunto de datos de clasificación multiclase sintético que podamos usar como base de la exploración.
2. Podemos usar la función `make_classification()` para definir un problema de clasificación multiclase con 1000 ejemplos, 10 características de entrada y 3 clases. 

In [1]:
# multi-class classification dataset
from sklearn.datasets import make_classification
from collections import Counter
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# summarize the dataset
print(X.shape, y.shape)
# summarize the number of examples in each class
print(Counter(y))

(1000, 10) (1000,)
Counter({1: 334, 2: 334, 0: 332})


<a id="section22"></a> 
## <font color="#004D7F"> 2.2. OvR con LoR </font>

A continuación, podemos demostrar la estrategia incorporada del algoritmo de clasificación binaria `LogisticRegression` para manejar más de dos clases. 
1. La estrategia para manejar la clasificación multiclase se puede establecer mediante el argumento `multi_class = "ovr"` para la estrategia OvR. 
2. El modelo se evaluará mediante una validación cruzada estratificada repetida de _k_ veces con 3 repeticiones y 10 veces.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> Más información sobre la clase [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) 
</div>

In [2]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class='ovr')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.686 (0.036)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section23"></a> 
## <font color="#004D7F"> 2.3. OvR con Scikit-learn </font>

La biblioteca scikit-learn también proporciona una clase `OneVsRestClassifier`. 
- Se puede utilizar para aprovechar un clasificador binario (e.g., LoR o MLP) para la clasificación multiclase
- O incluso otros clasificadores que admitan de forma nativa la clasificación multiclase.
- Se tiene que proporcionar a `OneVsRestClassifier` un clasificadorcomo argumento. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> Más información sobre la clase [`OneVsRestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) 
</div>

In [3]:
from sklearn.multiclass import OneVsRestClassifier

model = LogisticRegression()
ovr = OneVsRestClassifier(model)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(ovr, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.686 (0.036)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section24"></a> 
## <font color="#004D7F"> 2.4. OvR Predicciones </font>

También podemos utilizar la clase contenedora `OneVsRestClassifier` como modelo final y hacer predicciones. Para ello hay que llamar a la función `predict()` en nuevos datos. 

In [4]:
X[100]

array([ 1.62938294, -2.06916481,  0.81821172, -0.14754164,  0.09710822,
       -1.90502517, -0.10581739, -0.74460443, -1.4203441 , -0.86360262])

In [5]:
model = LogisticRegression()
ovr = OneVsRestClassifier(model)
ovr.fit(X, y)
row = [ 1.62938294, -2.06916481,  0.81821172, -0.14754164,  0.09710822,
       -1.90502517, -0.10581739, -0.74460443, -1.4203441 , -0.86360262]
predicted = ovr.predict([row])
print('Predicted Class: %d' % predicted)

Predicted Class: 0


---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section3"></a> 
# <font color="#004D7F"> 3. Uno contra uno para clasificación multiclase</font>

Al igual que One-vs-Rest, One-vs-One (OvO) divide un conjunto de datos de clasificación multiclase en problemas de clasificación binaria. 
- One-vs-Rest lo divide en un conjunto de datos binarios para cada clase
- One-vs-One divide el conjunto de datos para cada clase frente a todas las demás clases. 

Por ejemplo, considere un problema de clasificación multiclase con cuatro clases: roja, azul y verde, amarilla, sería:
- Problema de clasificación binaria 1: rojo versus azul
- Problema de clasificación binaria 2: rojo versus verde
- Problema de clasificación binaria 3: rojo versus amarillo
- Problema de clasificación binaria 4: azul versus verde
- Problema de clasificación binaria 5: azul versus amarillo
- Problema de clasificación binaria 6: verde versus amarillo

<a id="section31"></a> 
## <font color="#004D7F"> 3.1. OvO en SVM</font>

- De manera similar, la clase con la puntuación total más grande se predice como la etiqueta de clase. 
- `SVC` admite el método OvO para problemas de clasificación multiclase.
- Esto se puede lograr estableciendo el argumento `decision_function_shape='ovo'`. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> Más información sobre la clase [`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) 
</div>

In [6]:
from sklearn.svm import SVC

model = SVC(decision_function_shape = 'ovo')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.871 (0.028)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section32"></a> 
## <font color="#004D7F"> 3.2. OvO con Scikit-learn </font>

- La clase `OneVsOneClassifier` se puede utilizar con un clasificador binario como SVM, Regresión logística o Perceptron para clasificación multiclase.
- O incluso con otros clasificadores que admitan de forma nativa la clasificación multiclase. 
- `OneVsOneClassifier` requiere un clasificador como argumento. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> Más información sobre la clase [`OneVsOneClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsOneClassifier.html) 
</div>

In [7]:
from sklearn.multiclass import OneVsOneClassifier

model = SVC()
ovo = OneVsOneClassifier(model)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(ovo, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.872 (0.030)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section33"></a> 
## <font color="#004D7F"> 3.3. OvO Predicciones </font>

`OneVsOneClassifier` se puede usar como modelo final y hacer predicciones para la clasificación mediante la función `predict()`. 

In [8]:
from sklearn.multiclass import OneVsOneClassifier

model = SVC()
ovo = OneVsOneClassifier(model)
ovo.fit(X, y)
row = [ 1.62938294, -2.06916481,  0.81821172, -0.14754164,  0.09710822,
       -1.90502517, -0.10581739, -0.74460443, -1.4203441 , -0.86360262]
predicted = ovo.predict([row])
print('Predicted Class: %d' % predicted)

Predicted Class: 0


---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="sectionEj"></a>
<h3><font color="#004D7F" size=6> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicios</font></h3>

Se proponen las siguientes actividades para consolidar el aprendizaje.

# <font color="#004D7F" size=5>Ejercicio 1</font>
__Hiperparámetros__. ¿Qué otros hiperparámetros se pueden utilizar? Indaga y descubra más hiperparámetros y evalúe su uso.

### OneVsOneClassifier y OneVsRestClassifier

En el caso de `OneVsOneClassifier` y `OneVsRestClassifier` se podría añadir el siguiente hiperparámetro:
- `n_jobs` : El número de trabajos a usar para la computación en paralelo. None significa 1, y -1 significa usar todos los procesadores disponibles.

Además, en `OneVsRestClassifier` se puede usa también el hiperparámetro `verbose`.

Ninguno de ellos debería afectar el rendimiento del modelo.

### SVC

En `SVC` estos son los hiperparámetros principales. En negrita los más relevantes para el estudio:
- **`C`**: Parámetro de regularización. La fuerza de la regularización es inversamente proporcional a C. Un valor más pequeño especifica una regularización más fuerte.
- `kerne`l: Especifica el tipo de kernel a usar en el algoritmo. Los valores posibles son `'linear'`, `'poly'`, `'rbf'`, `'sigmoid'`, y `'precomputed'`.
- **`degree`**: Grado del kernel polinómico (solo se usa si `kernel='poly'`).
- **`gamma`**: Coeficiente del kernel para los kernels `'rbf'`, `'poly'` y `'sigmoid'`. Puede ser `'scale'`, `'auto'` o un valor flotante.
- `coef0`: Término independiente en el kernel. Solo es significativo en los kernels `'poly'` y `'sigmoid'`.
- `shrinking`: Si se debe usar la heurística de reducción. Es un valor booleano.
- **`probability`**: Si se deben habilitar las estimaciones de probabilidad. Esto debe habilitarse antes de llamar a fit.
- `tol`: Tolerancia para el criterio de parada.
- `cache_size`: Especifica el tamaño de la caché del kernel (en MB).
- **`class_weight`**: Peso asociado a cada clase. Puede ser un diccionario o `'balanced'`.
- `verbose`: Nivel de verbosidad. Si es mayor que 0, se imprimen mensajes de progreso.
- `max_iter`: Número máximo de iteraciones. Si es -1, no hay límite.
- **`decision_function_shape`**: Forma de la función de decisión. Puede ser `'ovr'` (uno contra el resto) o `'ovo'` (uno contra uno).
- **`break_ties`**: Si se deben romper los empates en la predicción. Solo es relevante si `decision_function_shape='ovr'`.
- `random_state`: Controla la aleatoriedad del clasificador

### Hiperparámetros de Logistic Regression (LoR)

En `LoR` estos son los hiperparámetros principales. En negrita los más relevantes para el estudio:

1. **penalty**: Especifica la norma de la penalización. Puede ser `'l1'`, `'l2'`, `'elasticnet'` o `None`. La penalización `'l2'` es la opción predeterminada.
2. **dual**: Booleano que indica si se debe usar la formulación dual. Solo se implementa para la penalización `'l2'` con el solver `liblinear`.
3. **tol**: Tolerancia para el criterio de parada.
4. **C**: Inverso de la fuerza de regularización; debe ser un flotante positivo. Valores más pequeños especifican una regularización más fuerte.
5. **fit_intercept**: Booleano que indica si se debe agregar una constante (intercepto) a la función de decisión.
6. **intercept_scaling**: Solo útil cuando se usa el solver `liblinear` y `fit_intercept` es `True`. Escala el intercepto.
7. **`class_weight`**: Peso asociado a cada clase. Puede ser un diccionario o `'balanced'`.
8. **random_state**: Controla la aleatoriedad del clasificador.
9. **solver**: Algoritmo a usar en el problema de optimización. Puede ser `'newton-cg'`, `'lbfgs'`, `'liblinear'`, `'sag'` o `'saga'`.
10. **max_iter**: Número máximo de iteraciones para los solvers de convergencia.
11. **`multi_class`**: Especifica si se debe usar `'ovr'` (uno contra el resto) o `'multinomial'` para problemas multiclase.
12. **verbose**: Nivel de verbosidad. Si es mayor que 0, se imprimen mensajes de progreso.
13. **warm_start**: Si se debe reutilizar la solución de la llamada anterior para ajustar el modelo.
14. **n_jobs**: Número de trabajos a usar para la computación en paralelo. `None` significa 1, y `-1` significa usar todos los procesadores disponibles.
15. **l1_ratio**: La mezcla de penalización L1 y L2. Solo se usa si `penalty='elasticnet'`.

# <font color="#004D7F" size=5>Ejercicio 2</font>
__Más conjuntos de datos__. Busque un dataset original y verdadero (que no sea sintético) y evalúe el uso de los conceptos vistos en esta unidad. Los conjuntos de datos en pueden ser obtenidos del [repositorio de aprendizaje automático de UCI](https://archive.ics.uci.edu/).

In [9]:
from sklearn.datasets import load_wine

# Cargar el dataset
data = load_wine()

# Ver las características del dataset
print(data.feature_names)

# Ver las etiquetas del dataset
print(data.target_names)

# Acceder a los datos y las etiquetas
X = data.data
y = data.target

# Imprimir la forma de los datos y las etiquetas
print(X.shape)
print(y.shape)

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
['class_0' 'class_1' 'class_2']
(178, 13)
(178,)


### OvR con LoR

In [10]:
model = LogisticRegression(multi_class='ovr')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.953 (0.053)


### OvR con OneVsRestClassifier

In [11]:
model = LogisticRegression()
ovr = OneVsRestClassifier(model)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(ovr, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.953 (0.053)


Para hacer predicciones:

In [12]:
y[3]

0

In [13]:
model = LogisticRegression(max_iter=1000)
ovr = OneVsRestClassifier(model)
ovr.fit(X, y)
row = X[5]
predicted = ovr.predict([row])
print('Predicted Class: %d' % predicted)

Predicted Class: 0


### OvO con SVC

In [14]:
model = SVC(decision_function_shape = 'ovo')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.687 (0.096)


### OvO con OneVsOneClassifier

In [15]:
model = SVC()
ovo = OneVsOneClassifier(model)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(ovo, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.710 (0.097)


Para hacer predicciones:

In [16]:
y[100]

1

In [17]:
model = SVC()
ovo = OneVsRestClassifier(model)
ovo.fit(X, y)
row = X[100]
predicted = ovo.predict([row])
print('Predicted Class: %d' % predicted)

Predicted Class: 1


# <font color="#004D7F" size=5>Ejercicio 3</font>
__Clasificadores__. Evalué todo los ejercicios anteriores con el uso de clasificadores distintos a los vistos en la unidad, i.e., diferentes a LoR, CART y NB.

### GaussianProcessClassifier
Vamos a probar los ejercicios haciendo uso de `GaussianProcessClassifier` y el dataset del Vino

In [18]:
from sklearn.gaussian_process import GaussianProcessClassifier

### OvR built-in

In [20]:
model = GaussianProcessClassifier(multi_class='one_vs_rest', n_jobs=-1)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.510 (0.100)


### OvR OneVersusRestClassifier

In [22]:
model = GaussianProcessClassifier()
ovr = OneVsRestClassifier(model)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(ovr, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.510 (0.100)


Precidiendo:

In [27]:
y[100]

1

In [28]:
model = GaussianProcessClassifier()
ovr = OneVsRestClassifier(model)
ovr.fit(X, y)
predicted = ovr.predict([X[100]])
print('Predicted Class: %d' % predicted)

Predicted Class: 1


### OvO built-in

In [29]:
model = GaussianProcessClassifier(multi_class='one_vs_one', n_jobs=-1)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.748 (0.106)


### OvO OneVersusOneClassifier

In [31]:
model = GaussianProcessClassifier()
ovo = OneVsOneClassifier(model)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(ovo, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Avg accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Avg accuracy: 0.748 (0.106)


Precidiendo:

In [35]:
y[177]

2

In [37]:
model = GaussianProcessClassifier()
ovo = OneVsOneClassifier(model)
ovo.fit(X, y)
predicted = ovo.predict([X[177]])
print('Predicted Class: %d' % predicted)

Predicted Class: 2


---

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>